<a href="https://colab.research.google.com/github/LeeSangMin96/music_classification/blob/master/NEW_baseline_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle -v

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 9.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 4.1 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=ce370aae86296681f97d3c687a5ca52e431140cd7ec7cfedc45e461d50234520
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Kaggle API 1.5.6


In [ ]:
!kaggle competitions download -c music-genres-classification 

  0% 0.00/404k [00:00<?, ?B/s]
100% 404k/404k [00:00<00:00, 109MB/s]


In [ ]:
!unzip music-genres-classification.zip

Archive:  music-genres-classification.zip
  inflating: submition_form.csv      
  inflating: test/blues/0_0.jpeg     
  inflating: test/blues/10_0.jpeg    
  inflating: test/blues/11_0.jpeg    
  inflating: test/blues/12_0.jpeg    
  inflating: test/blues/13_0.jpeg    
  inflating: test/blues/14_0.jpeg    
  inflating: test/blues/1_0.jpeg     
  inflating: test/blues/2_0.jpeg     
  inflating: test/blues/3_0.jpeg     
  inflating: test/blues/4_0.jpeg     
  inflating: test/blues/5_0.jpeg     
  inflating: test/blues/6_0.jpeg     
  inflating: test/blues/7_0.jpeg     
  inflating: test/blues/8_0.jpeg     
  inflating: test/blues/9_0.jpeg     
  inflating: test/classical/15_1.jpeg  
  inflating: test/classical/16_1.jpeg  
  inflating: test/classical/17_1.jpeg  
  inflating: test/classical/18_1.jpeg  
  inflating: test/classical/19_1.jpeg  
  inflating: test/classical/20_1.jpeg  
  inflating: test/classical/21_1.jpeg  
  inflating: test/classical/22_1.jpeg  
  inflating: test/classical/23

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
import random

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
trans = transforms.Compose([
    transforms.ToTensor(), #학습을 위해 dataset들을 텐서로 전환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = torchvision.datasets.ImageFolder(root='/content/train', transform=trans)
test_data = torchvision.datasets.ImageFolder(root='/content/test', transform=trans)

In [ ]:
batch_size = 200
data_loader = DataLoader(dataset = train_data, 
                         batch_size = batch_size, 
                         shuffle = True, 
                         num_workers=2, # 데이터를 빨리 읽어오기 위한것
                         drop_last=True)

In [ ]:
test_set = DataLoader(dataset = test_data, 
                      batch_size = len(test_data),
                      shuffle=False)

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1), 
            nn.MaxPool2d(kernel_size=2, stride=2), 
            nn.ReLU()
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), 
            nn.MaxPool2d(kernel_size=2, stride=2), 
            nn.ReLU()
        
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        

        self.fc = nn.Sequential(
            nn.Linear(128*4*4,256, bias=True),
            nn.ReLU(),
            nn.Linear(256, 10, bias=True)
        )    
        torch.nn.init.xavier_uniform_(self.fc[0].weight)
        torch.nn.init.xavier_uniform_(self.fc[2].weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [ ]:
model = MyModel().to(device)

In [ ]:
loss = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
total_batch = len(data_loader)
model.train()
for epoch in range(100):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        # one-hot encoding되어 있지 않음
        Y = Y.to(device)
      
        optimizer.zero_grad()
        hypothesis = model(X) #forward 계산
        cost = loss(hypothesis, Y) #cost
        cost.backward() #backpropagation
        optimizer.step() #갱신

        avg_cost += cost / total_batch #평균 error
    if epoch % 50==0:
        print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost,))
            

print('Learning finished')

Epoch: 0000 cost = 0.293123364
Epoch: 0050 cost = 0.220572203
Learning finished


In [ ]:
with torch.no_grad():
    model.eval()
    for _, data in enumerate(test_set):
        imgs, label = data
#                 print(len(imgs), len(label))
        imgs = imgs.to(device)
        label = label.to(device)

        prediction = model(imgs)

        correct_prediction = torch.argmax(prediction, 1) == label

        accuracy = correct_prediction.float().mean()
        print(accuracy.item())

0.36090224981307983


In [ ]:
import pandas as pd

submit=pd.read_csv('submition_form.csv')
submit

,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
128,128,0
129,129,0
130,130,0
131,131,0


In [ ]:
submit['label'] = torch.argmax(prediction, 1).cpu().numpy()
submit.to_csv('new_baseline.csv',index=False, header=True)

In [ ]:
!kaggle competitions submit -c music-genres-classification -f new_baseline.csv -m "submit"

100% 697/697 [00:04<00:00, 161B/s]
Successfully submitted to music_genres_classification